In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import gcsfs
pd.set_option('display.width', 1000)

### Find datasets which have a new version since a prior date
 - Specify the catalog date to compare to current catalog
 - Specify catalog type ('-noQC' or '') 

In [ ]:
fs = gcsfs.GCSFileSystem(token='anon',access='read_only',cache_timeout=-1)

date = '20210210'
cat_type = '-noQC'

# find closest catalog to specified date:
files = fs.glob(f'gs://cmip6/CMIP6/old_catalogs/pangeo-cmip6-*{cat_type}.csv.gz')
dates = [int(s.split('cmip6-')[-1].split(cat_type)[0]) for s in files]
prior_date = str(min(dates, key=lambda x:abs(x-int(date))))
prior_date

In [ ]:
# Read the catalogs
df_old = pd.read_csv(f'https://cmip6.storage.googleapis.com/CMIP6/old_catalogs/pangeo-cmip6-{prior_date}{cat_type}.csv.gz', dtype='unicode')
df_old['cstore'] = df_old.apply(lambda row: row.zstore.split('CMIP6/')[-1].split('v20')[0], axis = 1)
df = pd.read_csv(f'https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores{cat_type}.csv', dtype='unicode')
df['cstore'] = df.apply(lambda row: row.zstore.split('CMIP6/')[-1].split('v20')[0], axis = 1)

In [ ]:
dfn = pd.merge(df_old, df[['cstore','version']], on='cstore', how='left', sort=False, suffixes=('_old', '_new'))
df_replaced = dfn[dfn.version_old != dfn.version_new]
df_replaced = df_replaced[~(df_replaced.version_new.isnull())]
len(df_replaced), df_replaced.experiment_id.unique(), df_replaced.table_id.unique()

In [ ]:
df_replaced[['zstore', 'version_old','version_new']].style

In [ ]:
df.zstore.values[0]

In [ ]:
df_old.version.unique()